In [1]:
import xarray as xr
import rydanalysis as ra
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib
from pathlib import Path
import numpy as np
from tqdm.notebook import tqdm
from scipy.stats import sem
from scipy.integrate import quad

%matplotlib widget

In [2]:
height = 0.0005510203336598352

def load_peaks(live_analysis_path):
    live_analysis_path.mkdir(exist_ok=True)
    data = ra.load_data(path, lazy=True)
    
    peaks = data.scope_traces.peaks_summary.count(height=height, width=0.3*1e-9)
    ions_integrated = data.scope_traces.peaks_summary.integrate(height=height)
    ions = xr.merge([peaks, ions_integrated])
    # ions.ryd_data.to_netcdf(live_analysis_path / "peaks.h5")
    return ions

In [3]:
path = Path.cwd().parent / "raw_data"

live_analysis_path = Path.cwd().parent / "Analysis" 
peaks_file = live_analysis_path / "peaks.h5"

In [4]:
data = ra.load_data(path, to_multiindex=True, lazy=True)
variable = data.ryd_data.variables.columns[0]
variable

2021-01-27 17:54:57.937 INFO    numexpr.utils: NumExpr defaulting to 8 threads.


'dDDSmw'

In [6]:
prominence = 0
threshold = 0
width = 2e-9
distance = 0

# offset = float(traces_mi.sel({variable: 0}).mean("time").mean("tmstp"))
# std = float(traces_mi.sel({variable: 0}).std("time").mean("tmstp"))
# height = 3 * std + offset
height = 0.0007

In [7]:
peak_df = data.scope_traces.peaks_summary.get_peak_description(width=2e-9, height=height, distance=None)

In [8]:
peak_df.to_csv("peak_df.csv")

In [9]:
peak_df

,,,peak_heights,prominences,left_bases,right_bases,widths,width_heights,left_ips,right_ips,peak_time
peak_number,tmstp,dDDSmw,,,,,,,,,
0,2021-01-27 17:45:52,-20.0,0.000789,0.001604,0.000013,0.000019,3.975291e-09,-0.000013,0.000018,0.000018,0.000018
1,2021-01-27 17:45:52,-20.0,0.002592,0.003444,0.000013,0.000020,2.156738e-09,0.000870,0.000020,0.000020,0.000020
2,2021-01-27 17:45:52,-20.0,0.004520,0.005543,0.000013,0.000024,2.287319e-09,0.001749,0.000024,0.000024,0.000024
3,2021-01-27 17:45:52,-20.0,0.000707,0.001327,0.000024,0.000024,3.075804e-09,0.000044,0.000024,0.000024,0.000024
4,2021-01-27 17:45:52,-20.0,0.002781,0.004158,0.000024,0.000024,2.824388e-09,0.000702,0.000024,0.000024,0.000024
...,...,...,...,...,...,...,...,...,...,...,...
43,2021-01-27 17:51:46,20.0,0.003706,0.004979,0.000055,0.000063,2.724269e-09,0.001216,0.000057,0.000057,0.000057
44,2021-01-27 17:51:46,20.0,0.003481,0.004735,0.000057,0.000063,2.645000e-09,0.001113,0.000063,0.000063,0.000063
45,2021-01-27 17:51:46,20.0,0.000809,0.001801,0.000063,0.000063,4.149621e-09,-0.000092,0.000063,0.000063,0.000063


## Analysis

In [10]:
peak_df = pd.read_csv("peak_df.csv", index_col=[0, 1, 2])

In [11]:
summary = ra.summarize_peak_description(peak_df)
counts = summary.counts
counts_mean = counts.groupby(variable).mean()
counts_sem = counts.groupby(variable).reduce(sem)

In [12]:
from lmfit.models import GaussianModel, ConstantModel

gaussian_model = GaussianModel() + ConstantModel()
params = gaussian_model.make_params()
params.add("amplitude", 120, min=0)
params.add("center", 820)
params.add("c", 20)
params.add("sigma", 50)

gaussian_fit = gaussian_model.fit(counts_mean, x=counts_mean[variable], weights=1/counts_sem, params=params)

In [13]:
fig, ax = plt.subplots()

gaussian_fit.plot_fit(ax=ax)
ax.set_title(path.parent.name)
ax.set_xlabel(variable)
ax.set_ylabel("counts")

center = round(gaussian_fit.params["center"].value, 2)
center_err = round(gaussian_fit.params["center"].stderr, 2)

ax.annotate(r"center: ${} \pm {}$".format(center, center_err),
            (0.1, 0.9), xycoords="axes fraction")
fig.savefig("gaussian_fit.png", dpi=600, bbox_inches="tight")

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

TypeError: type NoneType doesn't define __round__ method

In [13]:
params

name,value,initial value,min,max,vary,expression
amplitude,120.000000,120,0.00000000,inf,True,
center,820.000000,820,-inf,inf,True,
sigma,50.0000000,50,-inf,inf,True,
c,20.0000000,20,-inf,inf,True,
fwhm,117.741000,None,-inf,inf,False,2.3548200*sigma
height,0.95746152,None,-inf,inf,False,"0.3989423*amplitude/max(2.220446049250313e-16, sigma)"


In [30]:
from lmfit import Model
def rabiFormular(delta,center, omega, t,off,amp):
    omega_eff = np.sqrt(omega**2 + (2 * (delta-center))**2)
    p_ee = amp * (omega**2/omega_eff**2 * np.sin(0.5 * 2 * np.pi * omega_eff * t)**2) + off
    return p_ee

model = Model(rabiFormular)

model.set_param_hint('omega', value = 15,vary = True)
model.set_param_hint('t', value = 0.1,vary = False)
model.set_param_hint('off', value = 40 , vary = True)
model.set_param_hint('amp', value = 200 , vary = True)
model.set_param_hint('center', value = 0 , vary = True)

params = model.make_params()

fitres = model.fit(counts_mean, delta=counts_mean[variable], weights=1/counts_sem, params=params)
fig, ax = plt.subplots()
fitres.plot_fit(ax=ax)

fig.savefig("fit_ramsey.png", dpi=600, bbox_inches="tight")

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [31]:
fitres

In [23]:
delta

NameError: name 'delta' is not defined

In [14]:
from lmfit import Model
def rabiFormular(delta,center, omega, t,off,amp):
    omega_eff = np.sqrt(omega**2 + (delta+center)**2)
    p_ee = amp * (omega**2/omega_eff**2 * np.sin(0.5 * 2 * np.pi * omega_eff * t)**2) + off
    return p_ee

model = Model(rabiFormular)

model.set_param_hint('omega', value = 80,vary = True)
model.set_param_hint('t', value = 0.017,vary = False)
model.set_param_hint('off', value = 10 , vary = True)
model.set_param_hint('amp', value = 70 , vary = True)
model.set_param_hint('center', value = -802.43 , vary = False)

params = model.make_params()

fitres = model.fit(counts_mean, delta=counts_mean[variable], weights=1/counts_sem, params=params)
fig, ax = plt.subplots()
fitres.plot_fit(ax=ax)

fig.savefig("fit_ramsey.png", dpi=600, bbox_inches="tight")

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [30]:
tqdm.pandas()

C:\Users\qd\Anaconda3\envs\rydanalysis\lib\site-packages\tqdm\std.py:706: FutureWarning: The Panel class is removed from pandas. Accessing it from the top-level namespace will also be removed in the next version
  from pandas import Panel


In [39]:
import time
def my_mean(df):
    time.sleep(0.01)
    return df.mean()

In [41]:
groupby = peak_df.groupby("oDDSmw").progress_aggregate(my_mean)

In [38]:
groupby.progress_aggregate